In [1]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from fredapi import Fred

In [2]:
#################################################################

# Base do Juros 10 anos.py
juros = pd.read_excel('/Users/guilhermerenatorosslerzanin/Antigo/Python/Wilson/all_rates.xlsx')
juros.drop(columns=['Unnamed: 0'], inplace=True)
juros.set_index('refdate', inplace=True)
req_cols = ['21 dias', '63 dias', '126 dias', '1 ano', '1,5 anos', '2 anos','2,5 anos', '3 anos', '3,5 anos', '4 anos', '5 anos', '6 anos', '6,5 anos', '7 anos', '7,5 anos', '8 anos', '8,5 anos', '9 anos', '9,5 anos', '10 anos', '10,5 anos', '11 anos', '11,5 anos', '12 anos', '20 anos', '30 anos']
juros.columns = req_cols
juros2 = juros[juros.columns[::-1]]  # Reverse the order of columns
juros2
juros3 = juros * 100
juros3 = juros3.dropna()
juros3


,21 dias,63 dias,126 dias,1 ano,"1,5 anos",2 anos,"2,5 anos",3 anos,"3,5 anos",4 anos,...,"8,5 anos",9 anos,"9,5 anos",10 anos,"10,5 anos",11 anos,"11,5 anos",12 anos,20 anos,30 anos
refdate,,,,,,,,,,,,,,,,,,,,,
2007-01-03,13.090022,12.809984,12.540012,12.304912,12.210000,12.164432,12.170000,12.163737,12.179998,12.180000,...,12.190000,12.170001,12.170001,12.170000,12.170000,12.170000,12.170000,12.170000,12.170000,12.170000
2007-01-04,13.090008,12.829621,12.579784,12.357860,12.299541,12.238388,12.250000,12.253491,12.290001,12.299998,...,12.327499,12.339999,12.339999,12.320000,12.320000,12.320000,12.320000,12.320000,12.320000,12.320000
2007-01-05,13.076271,12.836930,12.609969,12.433170,12.380000,12.326871,12.339999,12.340060,12.380000,12.376664,...,12.399999,12.409999,12.409999,12.390000,12.390000,12.390000,12.390000,12.390000,12.390000,12.390000
2007-01-08,13.050064,12.836885,12.628202,12.474672,12.436661,12.422925,12.420000,12.436746,12.469999,12.440002,...,12.469998,12.480001,12.480001,12.480000,12.480000,12.480000,12.480000,12.480000,12.480000,12.480000
2007-01-09,13.058715,12.839977,12.619231,12.458050,12.412499,12.423456,12.430002,12.434133,12.440000,12.449999,...,12.489999,12.489999,12.489999,12.480000,12.480000,12.480000,12.480000,12.480000,12.480000,12.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-27,13.540012,14.114999,14.512995,14.839997,14.878997,14.777004,14.722003,14.685003,14.701000,14.732002,...,14.803000,14.748000,14.748000,14.746001,14.746001,14.678001,14.678001,14.634000,14.590997,14.590997
2025-02-28,13.553063,14.150022,14.635010,15.040001,15.076996,15.033003,15.008002,14.989003,15.012000,15.057002,...,15.126999,15.071999,15.071999,15.050001,15.050001,14.982001,14.982001,14.937999,14.894998,14.894998
2025-03-05,13.580938,14.148017,14.545993,14.860006,14.869995,14.770996,14.731002,14.711002,14.731002,14.756001,...,14.834000,14.773998,14.773998,14.758999,14.758999,14.691000,14.691000,14.647002,14.603997,14.603997


In [3]:
def calculate_metrics(index_ticker, window, lookback_period=None):
  # Import reference interest rate
  rates = juros2[["10 anos"]]
  rates.columns = ['10-Year BR Treasury Yield (%)']

  # Import reference market index
  tickers = {'S&P500':'^GSPC',
             'S&P500 EW':'RSP',
            'Vanguard TMI':'VTI',
            'Russel 2000':'^RUT',
            'Ibovespa': '^BVSP'}
  market_index = pd.DataFrame(yf.download(tickers[index_ticker], start='1960-01-01', end='2024-12-31', auto_adjust=False)['Close'])
  market_index.columns = [index_ticker]

  # Combine dataframes
  rates_vs_market = rates.merge(market_index, how='left', left_index=True, right_index=True)
  
  # Adjust data to lookback period
  if lookback_period is not None:
    rates_vs_market = rates_vs_market[rates_vs_market.index.year >= (datetime.date.today().year - lookback_period)]

  # Resample data
  rates_vs_market = rates_vs_market.resample('M').last()

  # Calculate rates and market changes
  rates_vs_market['Yield Change'] = 100*(rates_vs_market['10-Year BR Treasury Yield (%)'] - rates_vs_market['10-Year BR Treasury Yield (%)'].shift(1))
  rates_vs_market['Window Yield Change'] = rates_vs_market['Yield Change'].rolling(window).sum()
  rates_vs_market['Window Market Returns'] = 100*rates_vs_market[index_ticker].pct_change(window)
  rates_vs_market['Following Window Market Returns'] = rates_vs_market['Window Market Returns'].shift(-window)
  rates_vs_market['Previous Window Market Returns'] = rates_vs_market['Window Market Returns'].shift(window)
  rates_vs_market.dropna(inplace=True)

  return rates, market_index, rates_vs_market

In [4]:
# Run Function
index_ticker = 'Ibovespa'
window = 1
lookback_period = None

rates, market_index, rates_vs_market = calculate_metrics(index_ticker, window, lookback_period)
rates_vs_market

[*********************100%***********************]  1 of 1 completed
/var/folders/mh/_pshb1sx6g754h7j4k0yy3qc0000gn/T/ipykernel_6418/269045884.py:23: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rates_vs_market = rates_vs_market.resample('M').last()
/var/folders/mh/_pshb1sx6g754h7j4k0yy3qc0000gn/T/ipykernel_6418/269045884.py:28: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rates_vs_market['Window Market Returns'] = 100*rates_vs_market[index_ticker].pct_change(window)


,10-Year BR Treasury Yield (%),Ibovespa,Yield Change,Window Yield Change,Window Market Returns,Following Window Market Returns,Previous Window Market Returns
refdate,,,,,,,
2007-03-31,0.11560,45805.0,-0.435000,-0.435000,4.358425,6.879162,-1.680032
2007-04-30,0.10565,48956.0,-0.994997,-0.994997,6.879162,6.765259,4.358425
2007-05-31,0.10025,52268.0,-0.540002,-0.540002,6.765259,4.063672,6.879162
2007-06-30,0.10650,54392.0,0.625001,0.625001,4.063672,-0.384248,6.765259
2007-07-31,0.11070,54183.0,0.420000,0.420000,-0.384248,0.837901,4.063672
...,...,...,...,...,...,...,...
2024-08-31,0.12040,136004.0,0.120999,0.120999,6.542788,-3.079321,3.022428
2024-09-30,0.12320,131816.0,0.279999,0.279999,-3.079321,-1.595406,6.542788
2024-10-31,0.12730,129713.0,0.410001,0.410001,-1.595406,-3.118423,-3.079321


In [5]:
# Plot rates behavior
fig = px.line(rates,
           x=rates.index,
           y=['10-Year BR Treasury Yield (%)'],
                 width=900)

fig.update_layout(title='10-Year BR Treasury Yield (%)',
    xaxis_title="Date",
    yaxis_title="Yield (%)",
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [6]:
# Plot rates change
fig = px.line(rates_vs_market,
           x=rates_vs_market.index,
           y=['Yield Change'],
                 width=900)

fig.update_layout(title='10-Year BR Treasury Yield Change (bps)',
    xaxis_title="Date",
    yaxis_title="Monthly Yield Change (bps)",
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [7]:
# Plot reference index
fig = px.line(market_index,
           x=market_index.index,
           y=index_ticker,
                 width=900)

fig.update_layout(title=f"{index_ticker} Index",
    xaxis_title="Date",
    yaxis_title="Index",
    legend_title="Legend")

fig.show()

In [8]:
# Plot bps changes in histogram
fig = px.histogram(rates_vs_market,
                   x=['Yield Change'],
                   marginal='box',
                   width=900)

fig.update_layout(title='10-Year BR Treasury Yield bps change',
    legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [9]:
for column in ['Window Market Returns', 'Following Window Market Returns', 'Previous Window Market Returns']:

    # Fit OLS regression model
    model = sm.OLS(rates_vs_market[column], sm.add_constant(rates_vs_market['Window Yield Change'])).fit()
    alpha = model.params.iloc[0]
    beta = model.params.iloc[1]
    rates_vs_market['Regression'] = alpha + (beta*rates_vs_market[column])

    # Plot rates vs market
    fig = px.scatter(rates_vs_market,
            x='Window Yield Change',
            y=[column, 'Regression'],
                    width=900)

    fig.update_layout(title=f"Yield Changes vs {column}",
        xaxis_title="Yield Change (bps)",
        yaxis_title="Market Returns (%)",
        legend=dict(title='Legend',
                                orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1))

    fig.show()

In [19]:
# Create subplots with different windows
window_list = [3, 6, 12, 24]
lookback_period = None
results_current_window = {}
results_following_window = {}
results_previous_window = {}
fig = make_subplots(rows=2, cols=2)

for window in window_list:

  # Calculate metrics
  rates, market_index, rates_vs_market = calculate_metrics(index_ticker, window, lookback_period)

  # Fit regression model
  #model = sm.OLS(rates_vs_market['Window Market Returns'], sm.add_constant(rates_vs_market['Window Yield Change'])).fit()
  #alpha = model.params[0]
  #beta = model.params[1]
  #rates_vs_market['Regression'] = alpha + (beta*rates_vs_market['Window Market Returns'])
  
  # Create chart
  trace_current = go.Scatter(
      x=rates_vs_market['Window Yield Change'],
      y=rates_vs_market['Window Market Returns'],
      mode='markers',
      marker=dict(size=4))
  results_current_window[window] = trace_current

  trace_following = go.Scatter(
      x=rates_vs_market['Window Yield Change'],
      y=rates_vs_market['Following Window Market Returns'],
      mode='markers',
      marker=dict(size=4))
  results_following_window[window] = trace_following

  trace_previous = go.Scatter(
      x=rates_vs_market['Window Yield Change'],
      y=rates_vs_market['Previous Window Market Returns'],
      mode='markers',
      marker=dict(size=4))
  results_previous_window[window] = trace_previous

# Create subplots
fig_current = make_subplots(rows=2, cols=2,
                    subplot_titles=("Janela de 3 Meses", "Janela de 6 Meses", "Janela de 12 Meses", "Janela de 24 Meses"))

fig_current.add_trace(results_current_window[3], row=1, col=1)
fig_current.add_trace(results_current_window[6], row=1, col=2)
fig_current.add_trace(results_current_window[12], row=2, col=1)
fig_current.add_trace(results_current_window[24], row=2, col=2)

# Update xaxis properties
fig_current.update_xaxes(title_text="Variação nos Juros (bps)", row=1, col=1)
fig_current.update_xaxes(title_text="Variação nos Juros (bps)", row=1, col=2)
fig_current.update_xaxes(title_text="Variação nos Juros (bps)", row=2, col=1)
fig_current.update_xaxes(title_text="Variação nos Juros (bps)", row=2, col=2)

# Update yaxis properties
fig_current.update_yaxes(title_text="Variação do Ibovespa (%)", row=1, col=1)
fig_current.update_yaxes(title_text="Variação do Ibovespa (%)", row=1, col=2)
fig_current.update_yaxes(title_text="Variação do Ibovespa (%)", row=2, col=1)
fig_current.update_yaxes(title_text="Variação do Ibovespa (%)", row=2, col=2)

fig_current.update_layout(height=800, width=950, title_text="Variação dos Juros de 10 anos vs Retorno da Bolsa", showlegend=False)
fig_current.show()

# Create subplots
fig_following = make_subplots(rows=2, cols=2,
                    subplot_titles=("Janela de 3 Meses", "Janela de 6 Meses", "Janela de 12 Meses", "Janela de 24 Meses"))

fig_following.add_trace(results_following_window[3], row=1, col=1)
fig_following.add_trace(results_following_window[6], row=1, col=2)
fig_following.add_trace(results_following_window[12], row=2, col=1)
fig_following.add_trace(results_following_window[24], row=2, col=2)

# Update xaxis properties
fig_following.update_xaxes(title_text="Variação nos Juros (bps)", row=1, col=1)
fig_following.update_xaxes(title_text="Variação nos Juros (bps)", row=1, col=2)
fig_following.update_xaxes(title_text="Variação nos Juros (bps)", row=2, col=1)
fig_following.update_xaxes(title_text="Variação nos Juros (bps)", row=2, col=2)

# Update yaxis properties
fig_following.update_yaxes(title_text="Variação do Ibovespa (%)", row=1, col=1)
fig_following.update_yaxes(title_text="Variação do Ibovespa (%)", row=1, col=2)
fig_following.update_yaxes(title_text="Variação do Ibovespa (%)", row=2, col=1)
fig_following.update_yaxes(title_text="Variação do Ibovespa (%)", row=2, col=2)

fig_following.update_layout(height=800, width=950, title_text="Variação dos Juros de 10 anos vs Retornos da Bolsa no Futuro", showlegend=False)
fig_following.show()

# Create subplots
fig_previous = make_subplots(rows=2, cols=2,
                    subplot_titles=("Janela de 3 Meses", "Janela de 6 Meses", "Janela de 12 Meses", "Janela de 24 Meses"))

fig_previous.add_trace(results_previous_window[3], row=1, col=1)
fig_previous.add_trace(results_previous_window[6], row=1, col=2)
fig_previous.add_trace(results_previous_window[12], row=2, col=1)
fig_previous.add_trace(results_previous_window[24], row=2, col=2)

# Update xaxis properties
fig_previous.update_xaxes(title_text="Variação nos Juros (bps)", row=1, col=1)
fig_previous.update_xaxes(title_text="Variação nos Juros (bps)", row=1, col=2)
fig_previous.update_xaxes(title_text="Variação nos Juros (bps)", row=2, col=1)
fig_previous.update_xaxes(title_text="Variação nos Juros (bps)", row=2, col=2)

# Update yaxis properties
fig_previous.update_yaxes(title_text="Variação do Ibovespa (%)", row=1, col=1)
fig_previous.update_yaxes(title_text="Variação do Ibovespa (%)", row=1, col=2)
fig_previous.update_yaxes(title_text="Variação do Ibovespa (%)", row=2, col=1)
fig_previous.update_yaxes(title_text="Variação do Ibovespa (%)", row=2, col=2)

fig_previous.update_layout(height=800, width=950, title_text="Variação dos Juros de 10 anos vs Retorno da Bolsa do Passado", showlegend=False)
fig_previous.show()


[*********************100%***********************]  1 of 1 completed
/var/folders/mh/_pshb1sx6g754h7j4k0yy3qc0000gn/T/ipykernel_6418/269045884.py:23: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/var/folders/mh/_pshb1sx6g754h7j4k0yy3qc0000gn/T/ipykernel_6418/269045884.py:28: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

[*********************100%***********************]  1 of 1 completed
/var/folders/mh/_pshb1sx6g754h7j4k0yy3qc0000gn/T/ipykernel_6418/269045884.py:23: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/var/folders/mh/_pshb1sx6g754h7j4k0yy3qc0000gn/T/ipykernel_6418/269045884.py:28: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will b

In [11]:
import requests
import pandas as pd

# Import Selic
codigo = 432
url_banco_central = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=json'
dados_selic = requests.get(url_banco_central)
json_selic = dados_selic.json()
selic = pd.DataFrame(json_selic)
# Converter a coluna 'data' para datetime e definir como índice
selic['data'] = pd.to_datetime(selic['data'], format='%d/%m/%Y')
selic.set_index('data', inplace=True)
selic['Selic'] = pd.to_numeric(selic['valor']) / 100
selic = selic[['Selic']]  # Manter apenas a coluna relevante


# Import 10 year interest rates metrics
juros = pd.read_excel('/Users/guilhermerenatorosslerzanin/Antigo/Python/Wilson/all_rates.xlsx')
juros.drop(columns=['Unnamed: 0'], inplace=True)
juros.set_index('refdate', inplace=True)
req_cols = ['21 dias', '63 dias', '126 dias', '1 ano', '1,5 anos', '2 anos','2,5 anos', '3 anos', '3,5 anos', '4 anos', '5 anos', '6 anos', '6,5 anos', '7 anos', '7,5 anos', '8 anos', '8,5 anos', '9 anos', '9,5 anos', '10 anos', '10,5 anos', '11 anos', '11,5 anos', '12 anos', '20 anos', '30 anos']
juros.columns = req_cols
juros2 = juros[juros.columns[::-1]]  # Reverse the order of columns
ten_year_treasury_yield = juros2[["10 anos"]]
ten_year_treasury_yield.columns = ['10-Year BR Treasury Yield (%)']
ten_year_treasury_yield

# Import market index
market_index = pd.DataFrame(yf.download('^BVSP', start='1960-01-01', auto_adjust=False)['Adj Close'])
market_index.columns = ['Ibovespa']
market_index.index = pd.to_datetime(market_index.index)
market_index




[*********************100%***********************]  1 of 1 completed


,Ibovespa
Date,
1993-04-27,24.500000
1993-04-28,24.299999
1993-04-29,23.700001
1993-04-30,24.100000
1993-05-03,24.100000
...,...
2025-03-12,123864.000000
2025-03-13,125637.000000
2025-03-14,128957.000000


In [12]:
# Combine dataframes
fed_rates_vs_market = selic.merge(market_index, how='inner', left_index=True, right_index=True)
fed_rates_vs_market = fed_rates_vs_market.merge(ten_year_treasury_yield, how='inner', left_index=True, right_index=True)
fed_rates_vs_market.index = pd.to_datetime(fed_rates_vs_market.index)
fed_rates_vs_market = fed_rates_vs_market[fed_rates_vs_market.index.year > 2018]
fed_rates_vs_market['Ibovespa'] = fed_rates_vs_market['Ibovespa']/fed_rates_vs_market['Ibovespa'].iloc[0]
fed_rates_vs_market

,Selic,Ibovespa,10-Year BR Treasury Yield (%)
2019-01-02,0.0650,1.000000,0.094958
2019-01-03,0.0650,1.006065,0.095058
2019-01-04,0.0650,1.009109,0.094157
2019-01-07,0.0650,1.007548,0.094858
2019-01-08,0.0650,1.011207,0.094157
...,...,...,...
2025-02-27,0.1325,1.371237,0.147460
2025-02-28,0.1325,1.349262,0.150500
2025-03-05,0.1325,1.351987,0.147590
2025-03-06,0.1325,1.355404,0.148360


In [16]:
# Plot data
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=fed_rates_vs_market.index, y=fed_rates_vs_market['Selic'], name='Selic'), secondary_y=False)
fig.add_trace(go.Scatter(x=fed_rates_vs_market.index, y=fed_rates_vs_market['Ibovespa'], name='Ibovespa'), secondary_y=True)
fig.add_trace(go.Scatter(x=fed_rates_vs_market.index, y=fed_rates_vs_market['10-Year BR Treasury Yield (%)'], name='10-Year BR Treasury Yield (%)'), secondary_y=False)
fig.update_yaxes(title_text="Ibovespa Compounded Normalized Returns", secondary_y=True)
fig.update_layout(title='Selic vs Mercado',
                    xaxis_title='Date',
                    yaxis_title='Rate (%)',
                height=600,
                width=950,
                legend=dict(title='Legend',
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1))

fig.show()

In [14]:
fed_rates_vs_market.corr()

,Selic,Ibovespa,10-Year BR Treasury Yield (%)
Selic,1.000000,0.342389,0.854036
Ibovespa,0.342389,1.000000,0.376451
10-Year BR Treasury Yield (%),0.854036,0.376451,1.000000


In [15]:
# Fit OLS regression model
fed_rates_vs_market.dropna(inplace=True)
model = sm.OLS(fed_rates_vs_market['Ibovespa'], sm.add_constant(fed_rates_vs_market['10-Year BR Treasury Yield (%)'])).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Ibovespa   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     252.0
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           1.24e-52
Time:                        20:26:45   Log-Likelihood:                 939.06
No. Observations:                1528   AIC:                            -1874.
Df Residuals:                    1526   BIC:                            -1863.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                             0.9739      0.016     59.504      0.000       0.942       1.006
10-Year BR Treasury Yield (%)     2.4509      0.154     15.873      0.000       2.148       2.754
==============================================================================
Omnibus:                       36.933   Durbin-Watson:                   0.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.241
Skew:                          -0.393   Prob(JB):                     3.01e-09
Kurtosis:                       2.994   Cond. No.                         46.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""